## import 칸

In [2]:
# import 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import plotly.express as px

import warnings
warnings.simplefilter("ignore") # 엑셀 파일 불러오때 심플 오류 무시하는 기능

## 5대 범죄현황, folium 데이터 처리, 전처리 및 시각화

In [3]:
# 로우데이터 파일 불러오기
# 애초에 데이터를 많이 가공한 상태로 가져오면 수고를 덜 수 있음
df_1 = pd.read_excel("5대+범죄+발생현황_20221021174201.xlsx")
# 범위 밖에 있는 데이터를 지우기 위한 컬럼 및 인덱스 작업
# dr_col = ["2013", "2013.1", "2013.2", "2013.3", 
#           "2013.4", "2013.5", "2013.6", "2013.7", 
#           "2013.8", "2013.9", "2013.10", "2013.11"] #더 편한 방법은 없을까?
# df_1 = df_1.drop(columns = dr_col)
df_1 = df_1.drop(columns = "자치구별(1)", index = 0)

# 로우데이터에서 강간과 강제추행의 데이터만 가져오기 위한 작업
# 2010.*에서 *의 숫자만 바꿔주면 데이터 변형 가능
# 하지만 2020.10 데이터는 다른 방식으로 접근해야함 (float이 적용이 되지 않음)

y_sum = []
for x in range(7):
    yn = x + 4
    yy = str(2010 + int(yn))
    yy_1 = str(2010.1 + int(yn))
    y_sum.append(yy)
    y_sum.append(yy_1)

# str()을 사용하면 숫자를 문자열로 변경 가능
# str(y_sum)

# y_sum에 처음 인덱스에 "자치구별(2)"를 넣어주고 형식을 맞춘 후에
# df_1에 해당 컬럼값만 가져오게 하는 방법
y_sum.insert(0, "자치구별(2)")
df_2 = df_1[y_sum]

# 컬럼명을 정리하고 필요없는 인덱스는 드랍하고 리 인덱싱 작업
df_2.columns = ["자치구별", "2014년_발생", "2014년_검거", 
                "2015년_발생", "2015년_검거", "2016년_발생", "2016년_검거", 
                "2017년_발생", "2017년_검거", "2018년_발생", "2018년_검거", 
                "2019년_발생", "2019년_검거", "2020년_발생", "2020년_검거"]
df_2 = df_2.drop(index = 1)
df_2 = df_2.drop(index = 2)
df_2 = df_2.reset_index(drop = True)

# melt를 이용하여 데이터를 시각화하기에 좋도록 정리
df_3 = df_2.melt(id_vars = "자치구별", value_vars = df_2.columns[1:], 
                 value_name = "수치", var_name = "발생 / 검거")

# 파생변수를 만듬 (연도와 발생 / 검거에 대한 파생변수)
df_3["연도"] = pd.to_numeric(df_3["발생 / 검거"].str[:4], errors = 'coerce')
df_3["발생 / 검거"] = df_3["발생 / 검거"].str[6:]

# 그룹바이 하여 한눈에 파악하기 쉽게 처리 가능
df_4 = df_3.groupby(["연도", "자치구별", "발생 / 검거"])[["수치"]].mean()

# 소계가 있는 인덱스를 모두 드랍하는 방법
dr_ind = df_3[df_3["자치구별"] == "소계"].index
df_5 = df_3.drop(dr_ind)
df_5

# plotly histogram 시각화
# 단점은 facet_col을 사용하면 자치구별이 모두 보이지 않음
# !!! 그룹바이 한 값은 row가 하나로 잡히기 때문에 시각화 데이터로 쓰기가 매우 어렵다.
# 시각화 데이터로 사용이 용이한것은 melt한 데이터들이다.
# px.histogram(df_5, x = "자치구별", y = "수치", color = "발생 / 검거", histfunc = "avg")

---------------------------------------------

import folium
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import plotly.express as px
from folium.plugins import MarkerCluster

import warnings
warnings.simplefilter("ignore") # 엑셀 파일 불러오때 심플 오류 무시하는 기능


# 지도에 기본 설정으로 찍힐 좌표를 설정하는 단계
# 결과 첫 화면 위치가 된다.
# location 에는 위도와 경도를 zoom start는 지정한 좌표에서 얼마나 확대된 화면이 출력되는 지를 지정하는 것, 범위는 0 ~ 18
# width 와 height 옵션도 있어서 지도의 크기 설정이 가능하다.
# title 의 디폴트 값은 OpenStreetMap이며, Stamen Toner와 cartodbpositron가 있다.
m = folium.Map(location = [37.56624759994501, 126.978844845852], zoom_start = 11, tiles="OpenStreetMap")


# 파일로 저장하기 위해서는 m.save(("경로/파일명.html")) 을 통해서 가능하다.
# 경로는 따로 설정하지 않으면 해당 파이썬 파일이 저장된 폴더로 디폴트 값되며, 기본 툴은 구글 맵으로 지정된다.
# m.save(("foli_map_ex.html")) -- 왼쪽 코드를 통해 파일 저장이 가능하다.

# 서울 행정구역 json raw파일(githubcontent)
# 변수로 requests 값 지정하기
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content

# json 모듈을 사용하여 html 형식의 데이터로 변경하기
seoul_geo = json.loads(c)

# json을 통해서 boundary를 표현하는 코드
# folium.GeoJson(data값(디폴트 값으로 그냥 데이터 값만 넣어도 출력됨), name = "해당 boundry의 이름")
folium.GeoJson(seoul_geo, name='지역구').add_to(m)

# 처리된 데이터를 불러오기
df = pd.read_excel("강간추행_데이터후처리파일.xlsx")
# 데이터의 형식이 궁금하다면 head를 통해 확인
# df.head()

# 자치구별의 이름을 넣었을 때, 해당 수치를 전부 더한값을 구해주는 함수 만들기
def de_sum(x):
    gu_sum = df[df["자치구별"] == x]["수치"].sum()
    return gu_sum

# 자치구별 컬럼 안에 중복치 아닌 값들을 전부 모아서 list 형식화
gu_list = list(df["자치구별"].unique())

# 딕셔너리 컨프리헨션을 통해 자치구와 de_sum 함수값들을 딕셔너리 형식화
gu_dict = {x : de_sum(x) for x in gu_list}

# 딕셔너리를 시리즈 변형하면 key 값은 인덱스로 value 값은 해당 value 값으로 변형된다.
gu_dict_sz = pd.Series(gu_dict)


# bins = list(gu_dict_sz.quantile([0, 0.25, 0.5, 0.75, 1]))
# bins 는 알아서 추가가 되었다? -- 공식 문서를 확인하고 추가 학습이 필요함

# 4분위수에 의한 색깔로 맵 표현하기
# 다른 parameter 들을 추가 학습이 필요함 / 공식 문서를 참고해야 함
m.choropleth(geo_data=seoul_geo,
             data=gu_dict_sz, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 강간, 강제추행 발생수 (2014~2020년도)")

# 원하는 좌표가 입력되있는 좌표 데이터 불러오기
seo_map = pd.read_excel("서울시_좌표.xlsx")

# 이전에 만들었던 자치구별 해당 범죄 발생수 데이터를 사용
# "자치구별"과 "발생수"라는 컬럼을 만들어 주기 위해 key 값으로 배정함
A = list(gu_dict.keys())
B = list(gu_dict.values())
gu_df = {"자치구별" : A, "발생수" : B}

# 위에서 만든 딕셔너리를 DataFrame 형식화
# columns = ["자치구별", "발생수"]
gu_df2 = pd.DataFrame(gu_df)

# 위 데이터와 서울시 좌표 데이터를 merge를 통해 합치기
# "on" parameter를 이용해서 "자치구별" 값이 맞아 떨어지도록 지정
gu_mer = pd.merge(left = gu_df2, right = seo_map, on = "자치구별")


# 맵에 숫자로 표시하는 기능(MarkerCluster() 메서드를 사용)
# from folium.plugins import MarkerCluster 모듈을 실행시켜야 함

marker_cluster = MarkerCluster().add_to(m)
for lat, long in zip(gu_mer["위도"], gu_mer["경도"]):
    folium.Marker([lat, long], tooltip = gu_mer[gu_mer["위도"] == lat][["자치구별", "발생수"]], icon = folium.Icon(color="green")).add_to(marker_cluster)
   
"""
tooltip =
f'Accepted<br />
Name: {df.iloc[i,0]}<br />
Cost: {df.iloc[i,3]:,.0f}<br /> 
Category: {df.iloc[i,4]}<br />
Distance: {np.round(df.iloc[i,6],2)}(m)<br />
Call payment: {df.iloc[i,5]}',

위 지도에서 좌표에 순수히 값만 나오게 하는 코드라 했지만, 이해를 못해서 우선 메모를 해둠
"""

,자치구별,발생 / 검거,수치,연도
1,종로구,발생,5021.0,2014
2,중구,발생,5231.0,2014
3,용산구,발생,3799.0,2014
4,성동구,발생,3582.0,2014
5,광진구,발생,6268.0,2014
...,...,...,...,...
359,관악구,검거,3858.0,2020
360,서초구,검거,3052.0,2020
361,강남구,검거,5245.0,2020
362,송파구,검거,3544.0,2020


## 5대 범죄현황, 도로 시설물, 전처리 및 시각화

In [6]:
#엑셀파일 불러오기
df = pd.read_csv("5대+범죄+발생현황_20221019163055.csv")

# 행 제거
df = df.drop([0,1,2,3],axis=0)

#이름 변경하기
df = df.rename(columns = {"자치구별(2)":"자치구별"})

df = df.melt(id_vars = "자치구별", value_vars = df.columns[1:], value_name = "건수", var_name = "년도")

df["건수"].replace('-',np.nan,inplace=True)

df = df.fillna(0)

df["건수"] = df["건수"].astype("int")

df["년"] = df["년도"].map(lambda x : int(x.split(".")[0]))
df["코드"] = df["년도"].map(lambda x : int(x.split(".")[1]))

kill_dict = {2: "발생", 3 :"검거"}
df["발생검거"] = df["코드"].map(kill_dict)

px.histogram(df, x ="건수", y ="자치구별",histfunc= "sum",color = "발생검거",barmode="group")

px.histogram(df,x = "건수", y="자치구별",histfunc="sum",color ="발생검거",facet_col ="발생검거")

plt.figure(figsize =(20,7))
sns.pointplot(data = df , x = "자치구별" ,y = "건수")

#발생검거 중 발생만 가져오기
df_C = df[df["발생검거"]=="발생"]
df_C.tail()

data_1 = df_C[df_C["년"] == 2014]
data_2 = df_C[df_C["년"] == 2015]
data_3 = df_C[df_C["년"] == 2016]
data_4 = df_C[df_C["년"] == 2017]
data_5 = df_C[df_C["년"] == 2018]
data_6 = df_C[df_C["년"] == 2019]
data_7 = df_C[df_C["년"] == 2020]

co_1 = data_1["건수"].sum()
co_2 = data_2["건수"].sum()
co_3 = data_3["건수"].sum()
co_4 = data_4["건수"].sum()
co_5 = data_5["건수"].sum()
co_6 = data_6["건수"].sum()
co_7 = data_7["건수"].sum()

n_1 = data_1["건수"] / co_1 * 100 
n_2 = data_2["건수"] / co_2 * 100 
n_3 = data_3["건수"] / co_3 * 100 
n_4 = data_4["건수"] / co_4 * 100
n_5 = data_5["건수"] / co_5 * 100
n_6 = data_6["건수"] / co_6 * 100
n_7 = data_7["건수"] / co_7 * 100

total = pd.concat([n_1,n_2,n_3,n_4,n_5,n_6,n_7])

total = total.rename("연도별 발생건수 백분율")

total_3 = pd.DataFrame(total)

df_A = pd.concat([df_C,total_3], axis =1)

pe = pd.read_csv("person.csv",encoding = "cp949")

pe = pe.drop("Unnamed: 0",axis =1)

df_join11 = pd.merge(df_A, pe, how='outer',  on="자치구별")

df_last = df_join11[["자치구별","생활인구 대비 폭력"]]

plt.figure(figsize=(20,15))
sns.barplot(data=df_last.sort_values("생활인구 대비 폭력", ascending=False), x="자치구별", y = "생활인구 대비 폭력")

---------------------------------------------------------

#데이터 불러오기
crime = pd.read_excel("5_20221018192142.xlsx",engine = "openpyxl")
crime.head(4)
# 데이터 크기 확인
crime.shape
#데이터 기본 정보 출력
crime.info()
#데이터 기술 통계 정보 요약
crime.describe()
# 고유값 개수
crime.value_counts()
#누락데이터 찾기
crime.isnull().sum().sum()
crime.columns
# 행 삭제 하기
crime = crime.drop(["장소별(1)","범죄별(1)","범죄별(2)","2012","2013","2021"],axis=1)
#일부 데이터 보기
crime.head(4)
# 행 삭제하기
crime = crime.drop([0,1,2,3,4,5],axis =0)
crime.head(5)
#결측지 값 제거
crime=crime.dropna()
#타입바꾸기
crime["연도"] = crime["연도"].astype("int")
crime["발생건수"] = crime["발생건수"].astype("int")
#타입 확인하기
crime.info()
#시각화
px.histogram(crime, x = "발생건수", y ="장소", histfunc='sum',title="장소별 발생건수")
crime_year = crime.groupby(["연도","장소"])["발생건수"].mean().unstack()
crime_year[["노상","단독주택","상점","유흥접객업소"]].plot(kind = "bar",figsize = (15,8),title="년도별 장소에 따른,범죄 증감률") 

FileNotFoundError: [Errno 2] No such file or directory: '5대+범죄+발생현황_20221021174201.csv'

## 5대 범죄현황, 고위험 음주율, 전처리 및 시각화

In [ ]:
# 1. 폭행 범죄 발생 데이터
# 1) 데이터를 불러와 쓸모 없는 열과 컬럼 제거
# * 2014 ~ 2020 컬럼을 제거한 것은 범죄 발생 합계 수치가 들어있는 컬럼이였기 때문
df_5crime = pd.read_csv(glob("data\\5대범죄*.csv")[1])
df_5crime = df_5crime.drop(columns="자치구별(1)", index=0)
df_5 = df_5crime.drop(columns=["2014", "2015", "2016", "2017", "2018", "2019", "2020"], index=[1, 2, 3])

# 2) 컬럼에 연도와 범죄 분류코드가 합쳐져 있어 melt로 데이터를 녹임
df_5m = df_5.melt(id_vars="자치구별(2)", var_name="연+분류", value_name="건수")

# 3) apply(lambda x : x.split(“.”)) 함수를 사용해 연도와 범죄 분류를 분리하여 
# 각각 연도와 분류코드 컬럼으로 넣어줌
df_5m["연도"] = df_5m["연+분류"].apply(lambda x : int(x.split(".")[0]))
df_5m["분류코드"] = df_5m["연+분류"].apply(lambda x : int(x.split(".")[1]))

# 4) 범죄 분류 코드별 범죄명을 딕셔너리에 넣어주고 map 함수로 매칭하여 
# 범죄 분류 코드를 ‘범죄명-발생’, ‘범죄명-검거’로 변환
crime_code ="""1 합계-검거
2 살인-발생
3 살인-검거
4 강도-발생
5 강도-검거
6 강간,강제추행-발생
7 강간,강제추행-검거
8 절도-발생
9 절도-검거
10 폭력-발생
11 폭력-검거"""
 
code_list = crime_code.split("\n")
code_dict = {int(n.split()[0]) : n.split()[1] for n in code_list}
df_5m["분류"] = df_5m["분류코드"].map(code_dict)

# 5) 분류 코드를 다시 분리하여 범죄종류와 발생/검거 컬럼으로 넣어줌
df_5m["범죄종류"] = df_5m["분류"].apply(lambda x : x.split("-")[0])
df_5m["발생/검거"] = df_5m["분류"].apply(lambda x : x.split("-")[1])

# 6) 범죄종류 중 폭력인 것을 가져와 새 데이터프레임으로 저장
df_hit = df_5m[df_5m["범죄종류"]=="폭력"]

# 7) 폭력 범죄 데이터프레임에서 "발생/검거" 컬럼의 값이 "발생"인 데이터만 가져와 
# groupby를 통해 자치구별 연평균 발생 건수만 저장함
hit_mean = round(df_hit[df_hit["발생/검거"]=="발생"].groupby(by=["자치구별"])["건수"].mean(), 2)


# 2. 고위험 음주율 데이터
# 1)데이터 로드하기
df_drink = pd.read_csv('data\\고위험+음주율_20221019142156.csv')

# 2)데이터 컬럼 및 인덱스 정리
df_drink = df_drink.drop(columns=["구분별(1)", "구분별(2)", "2014", "2015", "2016", "2017", "2018", "2019", "2020"], index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
df_drink_all = df_drink.drop(columns=["구분별(1)", "구분별(2)"], index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
df_drink_all = df_drink_all[["구분별(3)", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]]
df_drink_all = df_drink_all.rename(columns={"구분별(3)" : "자치구별"})
df_drink_all = df_drink_all.reset_index(drop=True)

# 3)melt로 데이터 녹여주기
df_drink_m = df_drink_all.melt(id_vars="자치구별", var_name="연도", value_name="음주율")

# 4)연도, 음주율 컬럼의 데이터 타입 바꿔주기
df_drink_m["연도"] = df_drink_m["연도"].astype(int)
df_drink_m["음주율"] = df_drink_m["음주율"].astype(float)

# 5)groupby를 통해 자치구별 연 평균 수치 구해주기
drink_mean = df_drink_m.groupby("자치구별")["음주율"].mean().sort_values(ascending=False)


# 3. 유흥주점 개수
# 1) 데이터 로드
df_sul = pd.read_csv('data\\서울시 유흥주점영업 인허가 정보.csv', encoding="cp949")

# 2) 정상 영업 중인 유흥주점 데이터만 가져오기
df_bar = df_sul[df_sul["영업상태명"] == "영업/정상"]

# 3) 결측치 제거
df_baraddress = df_bar["지번주소"].dropna()

# 4) 지번주소에서 자치구 데이터만 뽑아오기
df_baraddress = df_baraddress.apply(lambda x : x.split(" ")[1])

# 5) 자치구별 유흥주점 개수를 count하여 저장
df_bar_gu = df_baraddress.value_counts()


## 5대 범죄현황, CCTV 데이터, 생활인구 데이터, 전처리 및 시각화

In [ ]:
# 범죄 발생/검거 데이터(절도)
# 1) raw 데이터에서 필요 없는 index와 column 제거 및 column명 재설정
df = pd.read_csv("/Users/sangu/likelion_AIS/ais7/mid_project/mid_data/14~절도.csv")
df = df.drop([0,1,2,3],axis=0)
df = df.rename(columns = {"자치구별(2)" : "자치구"})
df = df.rename(columns = {"2014": "2014.1", "2014.1": "2014.2", "2015" : "2015.1", '2015.1' : "2015.2" ,'2016':"2016.1", '2016.1':"2016.2" ,'2017': "2017.1", '2017.1': "2017.2 ",'2018': "2018.1" ,'2018.1': "2018.2" ,'2019': "2019.1", '2019.1': "2019.2", '2020': '2020.1', '2020.1': '2020.2', '2021': '2021.1', '2021.1': '2021.2'})

# 2) melt 사용해서 데이터 보기 쉽게 바꾸기
df = df.melt(id_vars = "자치구", value_vars = df.columns[1:], value_name = "사건수", var_name = "년도")

# 3) 년도 column을 만들고 코드를 통해 발생과 검거를 나누는 column을 만듬
df["년"] = df["년도"].map(lambda x : int(x.split(".")[0]))
df["코드"] = df["년도"].map(lambda x : int(x.split(".")[1]))
kill = {1: "발생", 2 :"검거"}
df["발생검거"] = df["코드"].map(kill)

# 4) 시각화
# 4-1) column타입 변환하기
df["사건수"] = df["사건수"].astype(int)
plt.figure(figsize = (20,12))
sns.barplot(data = df, x= "자치구", y = "사건수", hue = "발생검거")
# sort_values()
sort_df = df.sort_values("사건수", ascending = False)
plt.figure(figsize = (20,12))
sns.barplot(data = sort_df, x= "자치구", y = "사건수", hue = "발생검거")
df[df["발생검거"] == "발생"].groupby("자치구")["사건수"].mean().sort_values(ascending = False).plot(kind = "bar", figsize = (20,12))

# cctv데이터

# 1) raw 데이터에서 필요 없는 index와 column 제거 및 column명 재설정
cctv = pd.read_csv("/Users/sangu/likelion_AIS/ais7/mid_project/mid_data/서울시CCTV설치운영현황(자치구)_연도별_220630기준.csv", encoding = "cp949")
cctv = cctv.drop(columns = {"2013년 이전\n설치된 CCTV"})
cctv = cctv.rename(columns = {"구분": "자치구"})
cctv = cctv.drop([0])

# 2) melt 사용해서 데이터 보기 쉽게 바꾸기
cctv = cctv.melt(id_vars = "자치구",value_vars = cctv.columns[2:], value_name = "cctv수", var_name = "년도")

# 3) 년도의 oooo"년" 년 빼기
cctv["년도"] = cctv["년도"].map(lambda x: x.replace("년", ""))
cctv["년도"].astype(int)

# 4) 범죄발생 수와 cctv concat
gangnam_cc = cctv[cctv["자치구"] == "강남구"]
gangnam_cc = gangnam_cc.reset_index(drop = True).drop([0]).reset_index(drop = True)
gangnam_event = gangnam[["사건수"]]
gangnam_event = gangnam_event.reset_index(drop = True)
concat_gangnam = pd.concat([gangnam_cc, gangnam_event], axis = 1)

# 전체 cctv 개수 보기
sort_cctv = cctv.sort_values("총계", ascending = False)
# 전체 cctv 개수
sort_cctv.plot(kind = "bar")

# 5) 시각화
fig = plt.figure(figsize=(10,6)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성
 
ax.plot(concat_gangnam['년도'],concat_gangnam['사건수']) ## 선그래프 생성

plt.xticks(rotation=45) ## x축 눈금 라벨 설정 - 40도 회전 
plt.title('사건수',fontsize=20) ## 타이틀 설정

fig = plt.figure(figsize=(10,5)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax_2 = ax.twinx() ## 그림 뼈대(프레임) 생성
 
ax.plot(concat_gangnam['년도'],concat_gangnam['cctv수']) ## 선그래프 생성

plt.xticks(rotation=45) ## x축 눈금 라벨 설정 - 40도 회전 
plt.title('cctv수',fontsize=20) ## 타이틀 설정
plt.show()

# 생활인구 데이터

# 1) raw 데이터 불러오기
df_gu = pd.read_csv("/Users/sangu/likelion_AIS/ais7/mid_project/mid_data/LOCAL_PEOPLE_GU_2021.csv", encoding = "cp949")

# 2) 자치구 코드와 자치구 맵함수 적용하기
# gu_list 에 자치구코드 : 자치구 로 묶었다.
gu_list = gu.split("\n")
gu_name = {}
for gl in gu_list:
    key = int(gl.split()[0])
    val = gl.split()[1]
    gu_name[key] = val
df_gu["구"] = df_gu["자치구코드"].map(gu_name)
df_gu = df_gu[['기준일ID', '시간대구분', '자치구코드', '구', '총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수','남자15세부터19세생활인구수', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수','남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '남자40세부터44세생활인구수','남자45세부터49세생활인구수', '남자50세부터54세생활인구수', '남자55세부터59세생활인구수','남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수', '여자0세부터9세생활인구수','여자10세부터14세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수','여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수','여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수','여자55세부터59세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수']]
df_gu[["자치구코드", "구"]]

# 3) column 삭제하기

df_gu = df_gu.drop(columns = {'남자0세부터9세생활인구수', '남자10세부터14세생활인구수','남자15세부터19세생활인구수', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수','남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '남자40세부터44세생활인구수','남자45세부터49세생활인구수', '남자50세부터54세생활인구수', '남자55세부터59세생활인구수','남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수', '여자0세부터9세생활인구수','여자10세부터14세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수','여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수','여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수','여자55세부터59세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수'})

# 4) 총생활인구수 컬럼 전처리하기
# 구별로 총생활인구수의 평균값 구하기
df_gu["총생활인구수"] = df_gu["총생활인구수"].astype(int)
test = list(df_gu["구"].unique())

test_2 = []
for x in test:
    test_1 = df_gu[df_gu["구"] == x]["총생활인구수"].mean()
    test_2.append((test_1))
person = pd.DataFrame(data = list(zip(test,test_2)), columns = ["구", "총생활인구수"])
person["총생활인구수"] = person["총생활인구수"].astype(float).round(1)
person["총생활인구수"] = person["총생활인구수"].astype(int)

# csv 저장
person.to_csv("person.csv", encoding="cp949")
# person < 생활인구

# 5) sort_values()
sort_person = person.sort_values("총생활인구수", ascending = False)


# 은영님 가로등 데이터에 인구수, 밀도 넣기

# 1) raw 데이터에서 필요 없는 index와 column 제거 및 column명 재설정
area_df = pd.read_csv("/Users/sangu/Downloads/행정구역별・지목별_국토이용현황_시군구_20221021113007.csv", encoding = "cp949")
area_df = area_df.drop([0,1])
area_df = area_df.drop(columns = {"시도(1)", "시군구(1)"})
area_df = area_df.rename(columns = {"2020": "면적"})
year_df = year_df.reset_index(drop = True)
area_df = area_df.reset_index(drop = True)

# 2) concat
concat_df = pd.concat([year_df, area_df], axis = 1)
concat_df["면적"] = concat_df["면적"].astype(float)
concat_df[["면적"]] = concat_df[["면적"]] / 1000000
concat_df = concat_df.rename(columns = {"면적": "면적:km"})
concat_df["건수"] = concat_df["건수"].astype(int)
concat_df["비교"] = concat_df["건수"] / concat_df["면적:km"]
concat_df[["비교"]] = concat_df[["비교"]].round(0).astype(int)

# 3) sort
sort_df = concat_df.sort_values("비교",ascending = False)

# person 데이터 불러오기
person = pd.read_csv("/Users/sangu/likelion_AIS/ais7/mid_project/person.csv", encoding = "cp949")

# raw 데이터 전처리
person = person.drop(columns = {"Unnamed: 0"})
person["총생활인구수"] = person["총생활인구수"].astype(float).round(1)
person["총생활인구수"] = person["총생활인구수"].astype(int)
person["총생활인구수"].head()

# 인덱싱
person = person[["총생활인구수"]]

# concat
df = pd.concat([concat_df, person], axis = 1)
df["밀도"] = df["총생활인구수"] / df["면적:km"]
df["밀도"] = df["밀도"].round(0).astype(int)
df.head()
df = df.rename(columns = {"건수": "가로등수" , "비교": "면적대비 가로등 비율", "총생활인구수": "인구수", "밀도": "밀도(명)"})

# sort
sort_person = df.sort_values("인구수", ascending = False)

# 시각화
plt.figure(figsize=(20, 8))
sns.barplot(data = sort_person, x = "자치구", y = "인구수")
sort_density = df.sort_values("밀도(명)", ascending = False)